In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import joblib
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, recall_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Dense, SimpleRNN, LSTM, Dropout, Bidirectional, TimeDistributed
from keras.layers.embeddings import Embedding

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def datasets(path):
  df = pd.read_csv(path)
  X_text = df['text'].values
  X_title = df['title'].values
  y = df['label'].values
  return X_text, X_title, y

In [ ]:
X_text1, X_title1, y1 = datasets('/content/drive/MyDrive/Group5_MLProject_EndSem/Dataset1.csv')

In [ ]:
X_text2, X_title2, y2 = datasets('/content/drive/MyDrive/Group5_MLProject_EndSem/Dataset3.csv')

In [ ]:
X_text3, X_title3, y3 = datasets('/content/drive/MyDrive/Group5_MLProject_EndSem/Dataset_4')

In [ ]:
embeddings_dict = {}
with open("/content/drive/MyDrive/MLProject/glove.840B.300d.txt", 'r') as f:
  for line in f:
    line = line.strip()
    values = line.split()
    word = values[0]
    try:
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_dict[word] = coefs
    except ValueError:
      pass 

In [ ]:
embeddings_dict = joblib.load('/content/drive/MyDrive/Group5_MLProject_EndSem/Glove_Embedding')

In [ ]:
def splitting(corpus, y):
  X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size=0.3, random_state=42)
  return X_train, X_test, y_train, y_test

In [ ]:
def preprocessing(X_train, X_test, embeddings_dict):
  t = Tokenizer()
  t.fit_on_texts(X_train)
  vocab_size = len(t.word_index)+1
  X_train_encoded = t.texts_to_sequences(X_train)
  X_test_encoded = t.texts_to_sequences(X_test)
  X_train_padded = pad_sequences(X_train_encoded,maxlen=150,padding='post')
  X_test_padded = pad_sequences(X_test_encoded,maxlen=150,padding='post')
  embeddings_matrix = np.zeros((vocab_size,300))
  for word,i in t.word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
      embeddings_matrix[i] = embedding_vector
  return embeddings_matrix, X_train_padded, X_test_padded, vocab_size

In [ ]:
def BiLSTM_model(embeddings_matrix, X_train_padded, y_train, X_test_padded, y_test, vocab_size):
  model = Sequential()
  model.add(Embedding(vocab_size,300,weights=[embeddings_matrix],input_length=150,trainable=False))
  model.add(Bidirectional(LSTM(600,return_sequences=True)))
  model.add(TimeDistributed(Dense(1,activation='sigmoid')))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  history = model.fit(X_train_padded,y_train,epochs=15,validation_data=(X_test_padded,y_test))
  return model

In [ ]:
def plotting_accuracies(test_accuracy_text,test_accuracy_title,test_accuracy_tt):
  l=[test_accuracy_text,test_accuracy_title,test_accuracy_tt]
  l2=['Text','Title','Text+Title']
  d1={'Accuracy':l,'Variation':l2}
  d1=pd.DataFrame(d1)
  import plotly.express as px

  fig = px.bar(d1, y='Accuracy', x='Variation', text='Accuracy',color='Variation',title='Accuracy Analysis on Dataset1')
  fig.update_traces(texttemplate='%{text}', textposition='outside')
  fig.show()

# Dataset1

In [ ]:
corpus1_text = joblib.load('/content/drive/MyDrive/Group5_MLProject_EndSem/Dataset1_Text_corpus')
corpus1_title = joblib.load('/content/drive/MyDrive/Group5_MLProject_EndSem/Dataset1_Title_corpus')

In [ ]:
X_train, X_test, y_train, y_test = splitting(corpus1_text, y1)

In [ ]:
embeddings_matrix, X_train_padded, X_test_padded, vocab_size = preprocessing(X_train, X_test, embeddings_dict)
model = BiLSTM_model(embeddings_matrix, X_train_padded, y_train, X_test_padded, y_test, vocab_size)

Epoch 1/15
455/455 [==============================] - 33s 73ms/step - loss: 0.5648 - accuracy: 0.7096 - val_loss: 0.4849 - val_accuracy: 0.7579
Epoch 2/15
455/455 [==============================] - 33s 72ms/step - loss: 0.4202 - accuracy: 0.8080 - val_loss: 0.3594 - val_accuracy: 0.8401
Epoch 3/15
455/455 [==============================] - 33s 73ms/step - loss: 0.2974 - accuracy: 0.8712 - val_loss: 0.3005 - val_accuracy: 0.8707
Epoch 4/15
455/455 [==============================] - 34s 74ms/step - loss: 0.2349 - accuracy: 0.9019 - val_loss: 0.2144 - val_accuracy: 0.9120
Epoch 5/15
455/455 [==============================] - 34s 74ms/step - loss: 0.1689 - accuracy: 0.9326 - val_loss: 0.1967 - val_accuracy: 0.9218
Epoch 6/15
455/455 [==============================] - 34s 75ms/step - loss: 0.1203 - accuracy: 0.9532 - val_loss: 0.2219 - val_accuracy: 0.9194
Epoch 7/15
455/455 [==============================] - 34s 75ms/step - loss: 0.0787 - accuracy: 0.9709 - val_loss: 0.2470 - val_accuracy:

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_padded,y_test)
print('Accuracy: ',test_accuracy)

195/195 [==============================] - 5s 26ms/step - loss: 0.4003 - accuracy: 0.9282
Accuracy:  0.9281935691833496


In [ ]:
# joblib.dump(test_accuracy, '/content/drive/MyDrive/Group5_MLProject_EndSem/Accuracy_BLSTM_D1')
test_accuracy_text1 = joblib.load('/content/drive/MyDrive/Group5_MLProject_EndSem/Accuracy_BLSTM_D1')

In [ ]:
X_train_title, X_test_title, y_train_title, y_test_title = splitting(corpus1_title, y1)
embeddings_matrix_title, X_train_title_padded, X_test_title_padded, vocab_size_title = preprocessing(X_train_title, X_test_title, embeddings_dict)
model_title = BiLSTM_model(embeddings_matrix_title, X_train_title_padded, y_train_title, X_test_title_padded, y_test_title, vocab_size_title)

Epoch 1/15
455/455 [==============================] - 33s 73ms/step - loss: 0.5736 - accuracy: 0.6535 - val_loss: 0.6609 - val_accuracy: 0.5924
Epoch 2/15
455/455 [==============================] - 33s 73ms/step - loss: 0.4505 - accuracy: 0.7761 - val_loss: 0.2594 - val_accuracy: 0.9103
Epoch 3/15
455/455 [==============================] - 33s 73ms/step - loss: 0.2536 - accuracy: 0.9121 - val_loss: 0.2540 - val_accuracy: 0.9109
Epoch 4/15
455/455 [==============================] - 33s 74ms/step - loss: 0.2515 - accuracy: 0.9127 - val_loss: 0.2558 - val_accuracy: 0.9103
Epoch 5/15
455/455 [==============================] - 34s 74ms/step - loss: 0.2476 - accuracy: 0.9138 - val_loss: 0.2562 - val_accuracy: 0.9110
Epoch 6/15
455/455 [==============================] - 34s 74ms/step - loss: 0.2478 - accuracy: 0.9138 - val_loss: 0.2567 - val_accuracy: 0.9103
Epoch 7/15
455/455 [==============================] - 34s 74ms/step - loss: 0.3028 - accuracy: 0.8662 - val_loss: 0.6602 - val_accuracy:

In [ ]:
test_loss, test_accuracy = model_title.evaluate(X_test_title_padded, y_test_title)
print('Accuracy: ',test_accuracy)

195/195 [==============================] - 5s 25ms/step - loss: 0.2183 - accuracy: 0.9178
Accuracy:  0.9177936315536499


In [ ]:
# joblib.dump(test_accuracy, '/content/drive/MyDrive/Group5_MLProject_EndSem/Accuracy_BLSTM_D1_title')
test_accuracy_title1 = joblib.load('/content/drive/MyDrive/Group5_MLProject_EndSem/Accuracy_BLSTM_D1_title')

In [ ]:
def concat_text_title(corpus_text, corpus_title):
  text_title = pd.DataFrame(zip(corpus_text, corpus_title))
  text_title.columns = ['text', 'title']
  text_title = text_title['text'].str.cat(text_title['title'], sep =" ")
  text_title = text_title.iloc[:].values
  return text_title

In [ ]:
text_title = concat_text_title(corpus1_title, corpus1_text)

In [ ]:
X_train_tt, X_test_tt, y_train_tt, y_test_tt = splitting(text_title, y1)

In [ ]:
embeddings_matrix_tt, X_train_tt_padded, X_test_tt_padded, vocab_size_tt = preprocessing(X_train_tt, X_test_tt, embeddings_dict)

In [ ]:
model_tt = BiLSTM_model(embeddings_matrix_tt, X_train_tt_padded, y_train_tt, X_test_tt_padded, y_test_tt, vocab_size_tt)

Epoch 1/15
455/455 [==============================] - 34s 74ms/step - loss: 0.5255 - accuracy: 0.7438 - val_loss: 0.4682 - val_accuracy: 0.7831
Epoch 2/15
455/455 [==============================] - 34s 74ms/step - loss: 0.4335 - accuracy: 0.7967 - val_loss: 0.3731 - val_accuracy: 0.8250
Epoch 3/15
455/455 [==============================] - 34s 74ms/step - loss: 0.2996 - accuracy: 0.8723 - val_loss: 0.2420 - val_accuracy: 0.8972
Epoch 4/15
455/455 [==============================] - 34s 75ms/step - loss: 0.2103 - accuracy: 0.9160 - val_loss: 0.2007 - val_accuracy: 0.9199
Epoch 5/15
455/455 [==============================] - 34s 75ms/step - loss: 0.1561 - accuracy: 0.9396 - val_loss: 0.1848 - val_accuracy: 0.9277
Epoch 6/15
455/455 [==============================] - 34s 76ms/step - loss: 0.1210 - accuracy: 0.9535 - val_loss: 0.1854 - val_accuracy: 0.9334
Epoch 7/15
455/455 [==============================] - 35s 76ms/step - loss: 0.0942 - accuracy: 0.9643 - val_loss: 0.1795 - val_accuracy:

In [ ]:
test_loss, test_accuracy = model_tt.evaluate(X_test_tt_padded, y_test_tt)
print('Accuracy: ',test_accuracy)

195/195 [==============================] - 5s 27ms/step - loss: 0.3400 - accuracy: 0.9388
Accuracy:  0.9387797713279724


In [ ]:
# joblib.dump(test_accuracy, '/content/drive/MyDrive/Group5_MLProject_EndSem/Accuracy_BLSTM_D1_tt')
test_accuracy_tt1 = joblib.load('/content/drive/MyDrive/Group5_MLProject_EndSem/Accuracy_BLSTM_D1_tt')

In [ ]:
plotting_accuracies(test_accuracy_text1,test_accuracy_title1,test_accuracy_tt1)

# Dataset2

In [ ]:
corpus2_text = joblib.load('/content/drive/MyDrive/Group5_MLProject_EndSem/Dataset2_Text_corpus')
corpus2_title = joblib.load('/content/drive/MyDrive/Group5_MLProject_EndSem/Dataset2_Title_corpus')

In [ ]:
X_train2, X_test2, y_train2, y_test2 = splitting(corpus2_text, y2)
embeddings_matrix2, X_train2_padded, X_test2_padded, vocab_size2 = preprocessing(X_train2, X_test2, embeddings_dict)
model2 = BiLSTM_model(embeddings_matrix2, X_train2_padded, y_train2, X_test2_padded, y_test2, vocab_size2)

Epoch 1/15
983/983 [==============================] - 72s 73ms/step - loss: 0.3625 - accuracy: 0.8474 - val_loss: 0.2744 - val_accuracy: 0.8958
Epoch 2/15
983/983 [==============================] - 73s 74ms/step - loss: 0.1590 - accuracy: 0.9383 - val_loss: 0.0880 - val_accuracy: 0.9682
Epoch 3/15
983/983 [==============================] - 74s 75ms/step - loss: 0.0661 - accuracy: 0.9771 - val_loss: 0.0640 - val_accuracy: 0.9779
Epoch 4/15
983/983 [==============================] - 74s 75ms/step - loss: 0.0437 - accuracy: 0.9850 - val_loss: 0.0473 - val_accuracy: 0.9824
Epoch 5/15
983/983 [==============================] - 74s 76ms/step - loss: 0.0293 - accuracy: 0.9902 - val_loss: 0.0435 - val_accuracy: 0.9850
Epoch 6/15
983/983 [==============================] - 75s 76ms/step - loss: 0.0187 - accuracy: 0.9942 - val_loss: 0.0406 - val_accuracy: 0.9874
Epoch 7/15
983/983 [==============================] - 74s 76ms/step - loss: 0.0115 - accuracy: 0.9965 - val_loss: 0.0472 - val_accuracy:

In [ ]:
test_loss, test_accuracy = model2.evaluate(X_test2_padded,y_test2)
print('Accuracy: ',test_accuracy)

421/421 [==============================] - 11s 26ms/step - loss: 0.0479 - accuracy: 0.9886
Accuracy:  0.9885739088058472


In [ ]:
# joblib.dump(test_accuracy, '/content/drive/MyDrive/Group5_MLProject_EndSem/Accuracy_BLSTM_D2')
test_accuracy_text2 = joblib.load('/content/drive/MyDrive/Group5_MLProject_EndSem/Accuracy_BLSTM_D2')

In [ ]:
X_train2_title, X_test2_title, y_train2_title, y_test2_title = splitting(corpus2_title, y2)
embeddings_matrix2_title, X_train2_title_padded, X_test2_title_padded, vocab_size2_title = preprocessing(X_train2_title, X_test2_title, embeddings_dict)
model2_title = BiLSTM_model(embeddings_matrix2_title, X_train2_title_padded, y_train2_title, X_test2_title_padded, y_test2_title, vocab_size2_title)

Epoch 1/15
983/983 [==============================] - 71s 73ms/step - loss: 0.6927 - accuracy: 0.5204 - val_loss: 0.6922 - val_accuracy: 0.5230
Epoch 2/15
983/983 [==============================] - 71s 72ms/step - loss: 0.6923 - accuracy: 0.5226 - val_loss: 0.6921 - val_accuracy: 0.5234
Epoch 3/15
983/983 [==============================] - 72s 73ms/step - loss: 0.6965 - accuracy: 0.5195 - val_loss: 0.7240 - val_accuracy: 0.4764
Epoch 4/15
983/983 [==============================] - 72s 73ms/step - loss: 0.6977 - accuracy: 0.5113 - val_loss: 0.6944 - val_accuracy: 0.4781
Epoch 5/15
983/983 [==============================] - 72s 73ms/step - loss: 0.6929 - accuracy: 0.5159 - val_loss: 0.6934 - val_accuracy: 0.5233
Epoch 6/15
983/983 [==============================] - 72s 73ms/step - loss: 0.6917 - accuracy: 0.5235 - val_loss: 0.6926 - val_accuracy: 0.5228
Epoch 7/15
983/983 [==============================] - 73s 74ms/step - loss: 0.6958 - accuracy: 0.5148 - val_loss: 0.6948 - val_accuracy:

In [ ]:
test_loss, test_accuracy = model2_title.evaluate(X_test2_title_padded, y_test2_title)
print('Accuracy: ',test_accuracy)

421/421 [==============================] - 11s 26ms/step - loss: 0.7318 - accuracy: 0.5221
Accuracy:  0.5221098065376282


In [ ]:
# joblib.dump(test_accuracy, '/content/drive/MyDrive/Group5_MLProject_EndSem/Accuracy_BLSTM_D2_title')
test_accuracy_title2 = joblib.load('/content/drive/MyDrive/Group5_MLProject_EndSem/Accuracy_BLSTM_D2_title')

In [ ]:
text_title2 = concat_text_title(corpus2_title, corpus2_text)

In [ ]:
X_train_tt2, X_test_tt2, y_train_tt2, y_test_tt2 = splitting(text_title2, y2)
embeddings_matrix_tt2, X_train_tt2_padded, X_test_tt2_padded, vocab_size_tt2 = preprocessing(X_train_tt2, X_test_tt2, embeddings_dict)
model_tt2 = BiLSTM_model(embeddings_matrix_tt2, X_train_tt2_padded, y_train_tt2, X_test_tt2_padded, y_test_tt2, vocab_size_tt2)

Epoch 1/15
983/983 [==============================] - 75s 76ms/step - loss: 0.2929 - accuracy: 0.8776 - val_loss: 0.1201 - val_accuracy: 0.9592
Epoch 2/15
983/983 [==============================] - 75s 76ms/step - loss: 0.0768 - accuracy: 0.9739 - val_loss: 0.0709 - val_accuracy: 0.9753
Epoch 3/15
983/983 [==============================] - 75s 76ms/step - loss: 0.0566 - accuracy: 0.9814 - val_loss: 0.0477 - val_accuracy: 0.9842
Epoch 4/15
983/983 [==============================] - 75s 76ms/step - loss: 0.0312 - accuracy: 0.9904 - val_loss: 0.0496 - val_accuracy: 0.9838
Epoch 5/15
983/983 [==============================] - 75s 76ms/step - loss: 0.0258 - accuracy: 0.9914 - val_loss: 0.0559 - val_accuracy: 0.9813
Epoch 6/15
983/983 [==============================] - 75s 76ms/step - loss: 0.0125 - accuracy: 0.9961 - val_loss: 0.0446 - val_accuracy: 0.9864
Epoch 7/15
983/983 [==============================] - 75s 76ms/step - loss: 0.0076 - accuracy: 0.9976 - val_loss: 0.0837 - val_accuracy:

In [ ]:
test_loss, test_accuracy = model_tt2.evaluate(X_test_tt2_padded, y_test_tt2)
print('Accuracy: ',test_accuracy)

In [ ]:
joblib.dump(test_accuracy, '/content/drive/MyDrive/Group5_MLProject_EndSem/Accuracy_BLSTM_D2_tt')
test_accuracy_tt2 = joblib.load('/content/drive/MyDrive/Group5_MLProject_EndSem/Accuracy_BLSTM_D2_tt')

In [ ]:
plotting_accuracies(test_accuracy_text2,test_accuracy_title2,test_accuracy_tt2)

# Datasets Accuracies Comparison and Analysis

In [ ]:
import plotly.graph_objects as go
l=[test_accuracy_text1,test_accuracy_title1,test_accuracy_tt1]
l1=[test_accuracy_text2,test_accuracy_title2,test_accuracy_tt2]
l2=['Text','Title','Text+Title']
fig = go.Figure(data=[
    go.Bar(name='Dataset1', x=l2, y=l,text=l,opacity=0.98),
    go.Bar(name='Dataset2', x=l2, y=l1,text=l1,opacity=0.98)
])
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(title_text='Accuracy Comparision on Datasets')
fig.update_layout(barmode='group')
fig.show()